In [ ]:
#!/usr/bin/env python3
"""
🏆 FINAL GOLD PIPELINE: Weighted 2-Model Ensemble
-------------------------------------------------
Architecture: Inception-SE + sEMG-Net (The Winner: 85.11%)
Strategy:     Class Weights to fix the remaining Class 1 vs 2 confusion.
Goal:         Break 86% Accuracy.
"""

import os
import glob
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, mixed_precision, callbacks
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import mode
from scipy.signal import butter, filtfilt, iirnotch
import warnings
import gc

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
DATA_DIR = 'data'
ARTIFACTS_DIR = 'artifacts_final'
FS = 512
EPOCHS = 60  # Increased slightly to allow class weights to settle
BATCH_SIZE = 128
RANDOM_SEED = 42
VAL_FILE_RATIO = 0.50
WINDOW_MS = 400
STRIDE_MS = 160
L2_REG = 1e-4

# GPU Setup
try:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print("✅ Mixed Precision (FP16) Enabled")
except:
    pass

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ==================== DATA & PREPROCESSING ====================
# (Standard Preprocessing Pipeline)

class SignalPreprocessor:
    def __init__(self, fs=1000, bandpass_low=20.0, bandpass_high=450.0, notch_freq=50.0):
        self.fs = fs
        nyq = fs / 2
        low = max(0.001, min(bandpass_low / nyq, 0.99))
        high = max(low + 0.01, min(bandpass_high / nyq, 0.999))
        self.b_bp, self.a_bp = butter(4, [low, high], btype='band')
        self.b_notch, self.a_notch = iirnotch(notch_freq, 30.0, self.fs) if notch_freq > 0 else (None, None)
        self.channel_means, self.channel_stds = None, None
        self.fitted = False

    def fit(self, signals_list):
        all_signals = np.concatenate(signals_list, axis=0)
        self.channel_means = np.mean(all_signals, axis=0)
        self.channel_stds = np.std(all_signals, axis=0) + 1e-8
        self.fitted = True
        return self

    def transform(self, signal):
        if len(signal) > 12:
            signal = filtfilt(self.b_bp, self.a_bp, signal, axis=0)
            if self.b_notch is not None:
                signal = filtfilt(self.b_notch, self.a_notch, signal, axis=0)
        if self.fitted:
            return (signal - self.channel_means) / self.channel_stds
        return (signal - np.mean(signal, axis=0)) / (np.std(signal, axis=0) + 1e-8)

    def segment(self, signal, window_ms=200, stride_ms=100):
        win_sz = int(window_ms * self.fs / 1000)
        step = int(stride_ms * self.fs / 1000)
        n = len(signal)
        if n < win_sz: return None
        n_win = (n - win_sz) // step + 1
        idx = np.arange(win_sz)[None, :] + np.arange(n_win)[:, None] * step
        return signal[idx]

def augment_dataset_advanced(X, y):
    print(f"    ⚡ Augmenting Data (Input: {len(X)} windows)...")
    b, t, c = X.shape

    # 1. Channel Masking
    X_mask = X.copy()
    mask_indices = np.random.choice(b, size=int(b * 0.5), replace=False)
    for i in mask_indices:
        ch = np.random.randint(0, c)
        X_mask[i, :, ch] = 0
    X_mask = X_mask + np.random.normal(0, 0.02, size=X_mask.shape)

    # 2. MixUp
    indices = np.random.permutation(b)
    X_shuffled = X[indices]
    alpha = 0.2
    lam = np.random.beta(alpha, alpha, size=(b, 1, 1))
    X_mix = lam * X + (1 - lam) * X_shuffled
    y_mix = y.copy()

    X_final = np.concatenate([X, X_mask, X_mix], axis=0)
    y_final = np.concatenate([y, y, y_mix], axis=0)
    print(f"    ⚡ Augmentation complete. Size: {len(X_final)} (3x)")
    return X_final, y_final

def get_session_files(data_dir, sessions):
    files = []
    for session in sessions:
        pattern = f'{data_dir}/**/{session}/**/*.csv'
        files.extend(sorted(glob.glob(pattern, recursive=True)))
    return files

def split_files_by_ratio(files, val_ratio, seed=RANDOM_SEED):
    gesture_files = {}
    for f in files:
        match = re.search(r'gesture(\d+)', f)
        if match:
            g = int(match.group(1))
            gesture_files.setdefault(g, []).append(f)
    train, val = [], []
    rng = random.Random(seed)
    for g, gfiles in gesture_files.items():
        rng.shuffle(gfiles)
        n_val = max(1, int(len(gfiles) * val_ratio))
        val.extend(gfiles[:n_val])
        train.extend(gfiles[n_val:])
    return train, val

def load_files_data(file_list):
    data_list, labels_list = [], []
    for f in file_list:
        try:
            lbl = int(re.search(r'gesture(\d+)', f).group(1))
            d = pd.read_csv(f).values
            if d.shape[1] >= 8:
                data_list.append(d)
                labels_list.append(np.full(len(d), lbl))
        except: pass
    return data_list, labels_list

def window_data(data_list, labels_list, prep, window_ms, stride_ms):
    X_wins, y_wins = [], []
    win_sz = int(window_ms * FS / 1000)
    step = int(stride_ms * FS / 1000)
    for d, l in zip(data_list, labels_list):
        d_filt = prep.transform(d)
        w = prep.segment(d_filt, window_ms, stride_ms)
        if w is not None:
            X_wins.append(w)
            n_win = (len(d) - win_sz) // step + 1
            idx = np.arange(win_sz)[None, :] + np.arange(n_win)[:, None] * step
            w_modes = mode(l[idx], axis=1, keepdims=True)[0].flatten()
            y_wins.append(w_modes)
    if not X_wins: return None, None
    return np.concatenate(X_wins), np.concatenate(y_wins)

# ==================== MODEL DEFINITIONS ====================

# 1. Inception-SE-TCN
def squeeze_excite_block(input_tensor, ratio=8):
    filters = input_tensor.shape[-1]
    se = layers.GlobalAveragePooling1D()(input_tensor)
    se = layers.Dense(filters // ratio, activation='relu', kernel_regularizer=l2(L2_REG))(se)
    se = layers.Dense(filters, activation='sigmoid', kernel_regularizer=l2(L2_REG))(se)
    se = layers.Reshape((1, filters))(se)
    return layers.Multiply()([input_tensor, se])

def inception_block(x, filters, dilation_rate):
    b1 = layers.Conv1D(filters//2, 3, dilation_rate=dilation_rate, padding='same', kernel_regularizer=l2(L2_REG))(x)
    b1 = layers.BatchNormalization()(b1)
    b1 = layers.Activation('relu')(b1)
    b2 = layers.Conv1D(filters//2, 7, dilation_rate=dilation_rate, padding='same', kernel_regularizer=l2(L2_REG))(x)
    b2 = layers.BatchNormalization()(b2)
    b2 = layers.Activation('relu')(b2)
    return layers.Concatenate()([b1, b2])

def make_inception_se_tcn(input_shape, n_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.GaussianNoise(0.05)(inputs)
    filters = 64
    for dilation_rate in [1, 2, 4, 8]:
        prev_x = x
        x = inception_block(x, filters, dilation_rate)
        x = layers.Dropout(0.2)(x)
        x = squeeze_excite_block(x, ratio=8)
        if prev_x.shape[-1] != filters:
            prev_x = layers.Conv1D(filters=filters, kernel_size=1, padding='same')(prev_x)
        x = layers.Add()([x, prev_x])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.MultiHeadAttention(key_dim=64, num_heads=4, dropout=0.3)(x, x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=l2(L2_REG))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(n_classes, activation='softmax', dtype='float32')(x)
    return keras.Model(inputs, outputs, name='Inception_SE_Attn')

# 2. sEMG Net
def conv_block(x, filters, kernel_size, pool=True):
    x = layers.Conv1D(filters, kernel_size, padding='same', kernel_regularizer=l2(L2_REG))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.25)(x)
    if pool: x = layers.MaxPooling1D(pool_size=2)(x)
    return x

def make_semg_net(input_shape, n_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.GaussianNoise(0.05)(inputs)
    x = conv_block(x, 64, 9, pool=False)
    x = conv_block(x, 128, 5, pool=True)
    x = conv_block(x, 256, 3, pool=True)
    x = conv_block(x, 512, 3, pool=True)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=l2(L2_REG))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(n_classes, activation='softmax', dtype='float32')(x)
    return keras.Model(inputs, outputs, name='sEMG_Net')

# ==================== 4. TRAINING LOGIC ====================

def train_and_save(model_builder, model_name, X_train, y_train_hot, X_val, y_val_hot, input_shape, n_classes, class_weights):
    print(f"\n🏋️ TRAIN: {model_name} (Weighted)")

    total_steps = len(X_train) // BATCH_SIZE * EPOCHS
    lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate=0.001, first_decay_steps=int(total_steps * 0.3),
        t_mul=2.0, m_mul=0.9, alpha=1e-5
    )

    save_path = f'{ARTIFACTS_DIR}/best_{model_name}.keras'
    checkpoint = callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=0)

    model = model_builder(input_shape, n_classes)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                  metrics=['accuracy'])

    # 🔥 APPLY CLASS WEIGHTS HERE
    model.fit(X_train, y_train_hot,
              validation_data=(X_val, y_val_hot),
              epochs=EPOCHS, batch_size=BATCH_SIZE,
              callbacks=[checkpoint],
              class_weight=class_weights,  # <--- The Key Fix
              verbose=1)

    tf.keras.backend.clear_session()
    gc.collect()
    return save_path

def main():
    os.makedirs(ARTIFACTS_DIR, exist_ok=True)
    print("="*70 + "\n🚀 FINAL GOLD PIPELINE: 2-Model Ensemble + Class Weights\n" + "="*70)

    # 1. Load Data
    existing_csvs = glob.glob(f'{DATA_DIR}/**/*.csv', recursive=True)
    if not existing_csvs:
        import gdown, zipfile
        gdown.download('https://drive.google.com/uc?id=16iNEwhThf2LcX7rOOVM03MTZiwq7G51x', 'dataset.zip', quiet=False)
        with zipfile.ZipFile('dataset.zip', 'r') as z: z.extractall(DATA_DIR)
        os.remove('dataset.zip')

    train_files = get_session_files(DATA_DIR, ['Session1', 'Session2'])
    session3_files = get_session_files(DATA_DIR, ['Session3'])
    val_files, test_files = split_files_by_ratio(session3_files, VAL_FILE_RATIO)

    train_data, train_labels = load_files_data(train_files)
    val_data, val_labels = load_files_data(val_files)
    test_data, test_labels = load_files_data(test_files)

    # 2. Preprocess
    prep = SignalPreprocessor(fs=FS).fit(train_data)
    X_train, y_train_raw = window_data(train_data, train_labels, prep, WINDOW_MS, STRIDE_MS)
    X_val, y_val_raw = window_data(val_data, val_labels, prep, WINDOW_MS, STRIDE_MS)
    X_test, y_test_raw = window_data(test_data, test_labels, prep, WINDOW_MS, STRIDE_MS)

    # 3. Augment
    X_train, y_train_raw = augment_dataset_advanced(X_train, y_train_raw)

    # 4. Encode
    le = LabelEncoder().fit(y_train_raw)
    y_train = le.transform(y_train_raw)
    y_val = np.array([le.transform([l])[0] if l in le.classes_ else -1 for l in y_val_raw])
    y_test = np.array([le.transform([l])[0] if l in le.classes_ else -1 for l in y_test_raw])

    n_classes = len(le.classes_)
    input_shape = X_train.shape[1:]

    y_train_hot = tf.keras.utils.to_categorical(y_train, n_classes)
    y_val_hot = tf.keras.utils.to_categorical(y_val, n_classes)

    # ==================== 5. DEFINE CLASS WEIGHTS ====================
    # Based on your confusion matrix: Class 1 and 2 are the weak points.
    class_weights = {
        0: 1.0,
        1: 1.5,  # Focus 50% more on Class 1
        2: 1.5,  # Focus 50% more on Class 2
        3: 1.0,
        4: 1.0
    }
    print(f"🎯 Strategy: Applying Class Weights: {class_weights}")

    # ==================== 6. TRAIN & SAVE ====================
    path_inception = train_and_save(make_inception_se_tcn, "inception_se",
                                    X_train, y_train_hot, X_val, y_val_hot, input_shape, n_classes, class_weights)

    path_semgnet = train_and_save(make_semg_net, "semg_net",
                                  X_train, y_train_hot, X_val, y_val_hot, input_shape, n_classes, class_weights)

    # ==================== 7. FINAL EVALUATION ====================
    print("\n" + "="*70 + "\n🎯 FINAL EVALUATION (2-Model Weighted)\n" + "="*70)

    m1 = make_inception_se_tcn(input_shape, n_classes)
    m1.load_weights(path_inception)

    m2 = make_semg_net(input_shape, n_classes)
    m2.load_weights(path_semgnet)

    p1 = m1.predict(X_test, batch_size=BATCH_SIZE, verbose=0)
    p2 = m2.predict(X_test, batch_size=BATCH_SIZE, verbose=0)

    # Simple Average (since models are equally strong)
    ensemble_probs = (p1 + p2) / 2.0
    ensemble_preds = ensemble_probs.argmax(axis=1)

    acc = accuracy_score(y_test, ensemble_preds)
    f1 = f1_score(y_test, ensemble_preds, average='macro')

    print(f"🏆 FINAL ACCURACY: {acc:.4f}")
    print(f"🏆 FINAL F1 SCORE: {f1:.4f}")

if __name__ == '__main__':
    main()